In [99]:
import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image
import torch
from torch.distributions import Categorical

images_base_path = "/Users/dfilipiak/repositories/publikacje/PUB-2021-OneShotKG/KGTN2/data/ILSVRC/Data/CLS-LOC"
base_path = "/Users/dfilipiak/repositories/publikacje/PUB-2021-OneShotKG/KGTN2/results/entropy"

paths = [ f"{base_path}/ResNet50_sgm-max-InnerProduct-{kgs}-{id}-1" for kgs in ("wiki,hierarchy,glove", ) for id in (1, 2, 5, 10) ]

In [100]:
results = []
for p in paths:
    
    feats = torch.from_numpy(np.load(f"{p}/features.npy"))
    weights = torch.from_numpy(np.load(f"{p}/weights.npy"))
    ys = torch.from_numpy(np.load(f"{p}/target.npy"))
    base_classes = torch.from_numpy(np.load(f"{p}/base_classes.npy")).tolist()
    novel_classes = torch.from_numpy(np.load(f"{p}/novel_classes.npy")).tolist()

    feats.shape, weights.shape, ys.shape, len(base_classes), len(novel_classes)

    output = torch.matmul(weights, feats)
    
    entropy2_novel = Categorical(logits = output[:, novel_classes]).entropy()
    entropy2 = Categorical(logits = output[:, base_classes + novel_classes]).entropy()
    
    en_mean_novel = entropy2_novel.mean(dim=0).mean().item()
    en_std_novel = entropy2_novel.std(dim=0).mean().item()    
    en_mean_all = entropy2.mean(dim=0).mean().item()
    en_std_all = entropy2.std(dim=0).mean().item()

    print(f'{p.rsplit("/", 1)[-1]}')
    results.append(f"{en_mean_novel:.2f} ($\pm${en_std_novel:.2f}) & {en_mean_all:.2f} ($\pm${en_std_all:.2f})")
print(" & ".join(results))
    # print(f"Entropy avg (mean over classes) on novel: {en_mean_novel:.2f}")
    # print(f"Entropy std (mean over classes) ona novel: {en_std_novel:.2f}")
    # print(f"Entropy avg (mean over classes) on all: {en_mean_all:.2f}")
    # print(f"Entropy std (mean over classes) ona all: {en_std_all:.2f}")

ResNet50_sgm-max-InnerProduct-wiki,hierarchy,glove-1-1
ResNet50_sgm-max-InnerProduct-wiki,hierarchy,glove-2-1
ResNet50_sgm-max-InnerProduct-wiki,hierarchy,glove-5-1
ResNet50_sgm-max-InnerProduct-wiki,hierarchy,glove-10-1
5.38 ($\pm$1.36) & 5.37 ($\pm$1.37) & 5.09 ($\pm$1.55) & 5.08 ($\pm$1.55) & 4.52 ($\pm$1.83) & 4.52 ($\pm$1.83) & 4.49 ($\pm$1.82) & 4.48 ($\pm$1.82)
